In [2]:
!python -m pip install --upgrade pynytimes

Requirement already up-to-date: pynytimes in c:\programdata\anaconda3\lib\site-packages (0.7.0)


In [49]:
from pynytimes import NYTAPI
import pandas as pd

nyt = NYTAPI("H99sTxFGH4V0ZAyfNA5RASCLM5I3WpH7", parse_dates=True)

import datetime



articles = nyt.article_search(
#     query = "Obama",
    results = 30,
    dates = {
        "begin": datetime.datetime(2021, 8, 20),
        "end": datetime.datetime(2021, 8, 24)
    },
    options = {
        "sort": "oldest",
        "sources": [
            "New York Times",
            "AP",
            "Reuters",
            "International Herald Tribune"
        ],
        "news_desk": [
            "Business"
        ],
        "type_of_material": [
            "News Analysis"
        ],
        "section_name": [
            "Business"
        ]
    }
    
)

print(articles)


[]


In [91]:
import datetime

articles = nyt.article_search(
#     query = "Obama",
#     results = 5,
    dates = {
        "begin": datetime.datetime(2021, 8, 23),
        "end": datetime.datetime(2021, 8, 23)
    },
    options = {
        "sort": "newest",
#         "sources": [
#             "New York Times"
#         ],
        "news_desk": [
            "Business"
        ]
#         "type_of_material": [
#             "News Analysis"
#         ]
    }
)


for i in range(len(articles)):
    print("-----------------------")
    print(articles[i]['abstract'])
    print(articles[i]['lead_paragraph'])
    print("-----------------------")


-----------------------
The law, the result of a $200 million proposition fight last year, ensures that workers like Uber and Lyft drivers are considered independent contractors.
Emboldened by a California election victory that maintained the independence of their drivers last year, gig economy companies like Uber and Lyft have in recent months accelerated a push for what they call a “third way” of working, a classification of independent gig workers who receive limited benefits without gaining employee status.
-----------------------
-----------------------
A proposal in the South Korean legislature is an early test of how forcefully the Biden administration will defend the companies abroad while trying to trim their power at home.
WASHINGTON — For months, Apple and Google have been fighting a bill in the South Korean legislature that they say could imperil their lucrative app store businesses. The companies have appealed directly to South Korean lawmakers, government officials and th

In [85]:
articles

[{'abstract': 'The company praised itself this week for being “the most transparent platform on the internet.”',
  'web_url': 'https://www.nytimes.com/2021/08/20/technology/facebook-popular-posts.html',
  'snippet': 'The company praised itself this week for being “the most transparent platform on the internet.”',
  'lead_paragraph': 'When Facebook this week released its first quarterly report about the most viewed posts in the United States, Guy Rosen, its vice president of integrity, said the social network had undertaken “a long journey” to be “by far the most transparent platform on the internet.” The list showed that the posts with the most reach tended to be innocuous content like recipes and cute animals.',
  'print_section': 'B',
  'print_page': '1',
  'source': 'The New York Times',
  'multimedia': [{'rank': 0,
    'subtype': 'xlarge',
    'caption': None,
    'credit': None,
    'type': 'image',
    'url': 'images/2021/08/20/business/20facebook/merlin_181026132_7bf264fa-1186-4

In [34]:
import os
import json
import time
import requests
import datetime
import dateutil
import pandas as pd
from dateutil.relativedelta import relativedelta

end = datetime.date.today()
start = end - relativedelta(years=1)

months_in_range = [x.split(' ') for x in pd.date_range(start, end, freq='MS').strftime("%Y %m").tolist()]

def send_request(date):
    '''Sends a request to the NYT Archive API for given date.'''
    base_url = 'https://api.nytimes.com/svc/archive/v1/'
    url = base_url + '/' + date[0] + '/' + date[1] + '.json?api-key=' + 'H99sTxFGH4V0ZAyfNA5RASCLM5I3WpH7'
    response = requests.get(url)
    print(response)
    time.sleep(6)
    return response


def is_valid(article, date):
    '''An article is only worth checking if it is in range, and has a headline.'''
    is_in_range = date > start and date < end
    has_headline = type(article['headline']) == dict and 'main' in article['headline'].keys()
    return is_in_range and has_headline


def parse_response(response):
    '''Parses and returns response as pandas data frame.'''
    data = {'headline': [],  
        'date': [], 
        'doc_type': [],
        'material_type': [],
        'section': [],
        'keywords': []}
    
    articles = response['response']['docs'] 
    for article in articles: # For each article, make sure it falls within our date range
        date = dateutil.parser.parse(article['pub_date']).date()
        if is_valid(article, date):
            data['date'].append(date)
            data['headline'].append(article['headline']['main']) 
            if 'section' in article:
                data['section'].append(article['section_name'])
            else:
                data['section'].append(None)
            data['doc_type'].append(article['document_type'])
            if 'type_of_material' in article: 
                data['material_type'].append(article['type_of_material'])
            else:
                data['material_type'].append(None)
            keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
            data['keywords'].append(keywords)
    return pd.DataFrame(data) 


def get_data(dates):
    '''Sends and parses request/response to/from NYT Archive API for given dates.'''
    total = 0
    print('Date range: ' + str(dates[0]) + ' to ' + str(dates[-1]))
    if not os.path.exists('headlines'):
        os.mkdir('headlines')
    for date in dates:
        response = send_request(date)
        df = parse_response(response)
        total += len(df)
        df.to_csv('headlines/' + date[0] + '-' + date[1] + '.csv', index=False)
        print('Saving headlines/' + date[0] + '-' + date[1] + '.csv...')
    print('Number of articles collected: ' + str(total))
    
    
get_data(months_in_range)

Date range: ['2020', '09'] to ['2021', '08']
<Response [403]>


TypeError: 'Response' object is not subscriptable

In [28]:
pd.date_range(start, end, freq='MS').strftime("%Y %m").tolist()

['2020 09',
 '2020 10',
 '2020 11',
 '2020 12',
 '2021 01',
 '2021 02',
 '2021 03',
 '2021 04',
 '2021 05',
 '2021 06',
 '2021 07',
 '2021 08']